## Covert notebook to raw python
To run on ML-Engine, we strip all cells tagged "display" to remove the dependency on matplotlib.
The remote server/cluster does not have a display anyway, just logs.
(To view/edit tags on notebook cells: View>Cell Toolbar>Tags)

In [44]:
# Convert notebook to raw python format and remove all cells tagged "display"
NOTEBOOK='01_RNN_generator_temperatures_solution.ipynb'
jupyter nbconvert tutorial/${NOTEBOOK} \
    --to python --TagRemovePreprocessor.remove_cell_tags={\"display\"} \
    --output task.py

[NbConvertApp] Converting notebook tutorial/01_RNN_generator_temperatures_solution.ipynb to python
[NbConvertApp] Writing 13240 bytes to tutorial/task.py


## To run on ML-Engine
First you will have to:
1. Create a Google Cloud Platform project
1. Enable billing
1. Create a Google Cloud Storage bucket (put in in region us-central1)
1. Enable the necessary APIs and request the necessary quotas

And fill in the info in the variables below.
You can try running on a GPU by using --scale-tier=BASIC_GPU or a CPU using --scale-tier=BASIC

In [45]:
BUCKET='ml1-demo-martin'
PROJECT='cloudml-demo-martin'

REGION='us-central1'
JOBNAME=sines_$(date -u +%y%m%d_%H%M%S)
OUTDIR="gs://${BUCKET}/sinejobs/${JOBNAME}"
gcloud ml-engine jobs submit training $JOBNAME \
 --region=$REGION \
 --module-name=tutorial.task \
 --package-path=tutorial \
 --job-dir=$OUTDIR \
 --scale-tier=BASIC_GPU \
 --runtime-version=1.6 \
 -- \
 --data-dir="gs://good-temperatures-public"

Job [sines_180405_190630] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe sines_180405_190630

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs sines_180405_190630
jobId: sines_180405_190630
state: QUEUED


## To test-run locally as if the code was running on ML-Engine
Warning: this will use the "python" command to run (usually mapped to python 2 on mac, same as ML-Engine)

In [12]:
gcloud ml-engine local train \
 --module-name=tutorial.task \
 --package-path=tutorial \
 --job-dir="checkpoints" \
 -- \
 --data-dir="gs://good-temperatures-public"

dyld: warning, LC_RPATH $ORIGIN/../../_solib_darwin_x86_64/_U_S_Stensorflow_Spython_C_Upywrap_Utensorflow_Uinternal.so___Utensorflow in /Users/mgorner/Library/Python/2.7/lib/python/site-packages/tensorflow/python/_pywrap_tensorflow_internal.so being ignored in restricted program because it is a relative path
Tensorflow version: 1.4.1
Reading data from "gs://good-temperatures-public".
Wrinting checkpoints to "outputdir".
Loading 8 files........
Pattern "gs://good-temperatures-public/*.csv" matches 1666 files
Pattern "gs://good-temperatures-public/USC000*2.csv" matches 8 files
Evaluation files: 8
Training files: 1658
Initial shape of the evaluation dataset: (8, 18262, 3)
8 files, 18262 data points per file, 3 values per data point (Tmin, Tmax, is_interpolated) 
Loading 8 files........
2018-04-03 23:21:30.801474: I tensorflow/core/platform/cpu_feature_guard.cc:137] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.2 AVX AVX2 FMA
